<a href="https://colab.research.google.com/github/LSzubelak/ls-projects/blob/main/500%2B_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install eurostat
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.9 MB/s eta 0:00:00


In [ ]:
import eurostat
import pycountry
import pandas as pd

In [ ]:
df_fertility = eurostat.get_data_df("tps00199")
df_fertility.head()

,freq,indic_de,geo\TIME_PERIOD,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,TOTFERRT,AL,1.71,1.73,1.73,1.59,1.54,1.48,1.37,NaN,1.34,1.31,1.21,NaN
1,A,TOTFERRT,AM,NaN,NaN,NaN,1.62,1.62,1.56,1.57,1.60,NaN,NaN,NaN,NaN
2,A,TOTFERRT,AT,1.44,1.44,1.46,1.49,1.53,1.52,1.47,1.46,1.44,1.48,1.41,1.32
3,A,TOTFERRT,AZ,2.00,1.98,1.97,1.94,1.90,1.75,1.73,1.81,NaN,NaN,NaN,NaN
4,A,TOTFERRT,BE,1.80,1.76,1.74,1.70,1.68,1.65,1.62,1.60,1.55,1.60,1.53,1.47


In [29]:
import re
import eurostat
import pandas as pd
import pycountry

# 1. Download the raw fertility table and reset its index
df_wide = eurostat.get_data_df("tps00199")
df = df_wide.reset_index()

# 2. Identify & rename the geo‑code column
geo_col = [c for c in df.columns if 'geo' in c.lower()][0]
df = df.rename(columns={geo_col: "geo"})

# 3. Drop unneeded dims
df = df.drop(columns=["freq", "indic_de"], errors="ignore")

# 4. Detect true year columns (exactly four digits)
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]

# 5. Melt into long form
df_long = df.melt(
    id_vars=["geo"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Fertility Rate"
)

# 6. Build a fallback for codes pycountry won’t find
_manual = {
    "EL": "Greece",
    "UK": "United Kingdom",
    "XK": "Kosovo",
    # add more here if you spot unmapped codes…
}

def code_to_name(code):
    if code in _manual:
        return _manual[code]
    try:
        return pycountry.countries.get(alpha_2=code).name
    except:
        return code  # leave as‑is if still not found

# 7. Map every code → country name
df_long["Country"] = df_long["geo"].apply(code_to_name)

# 8. Clean up types & drop missing
df_long["Year"] = df_long["Year"].astype(int)
df_clean = (
    df_long
    .dropna(subset=["Fertility Rate"])
    [["Country", "Year", "Fertility Rate"]]
    .sort_values(["Country","Year"])
    .reset_index(drop=True)
)

print(df_clean.head(10))


   Country  Year  Fertility Rate
0  Albania  2012            1.71
1  Albania  2013            1.73
2  Albania  2014            1.73
3  Albania  2015            1.59
4  Albania  2016            1.54
5  Albania  2017            1.48
6  Albania  2018            1.37
7  Albania  2020            1.34
8  Albania  2021            1.31
9  Albania  2022            1.21


In [30]:
df_clean

,Country,Year,Fertility Rate
0,Albania,2012,1.71
1,Albania,2013,1.73
2,Albania,2014,1.73
3,Albania,2015,1.59
4,Albania,2016,1.54
...,...,...,...
510,United Kingdom,2014,1.81
511,United Kingdom,2015,1.80
512,United Kingdom,2016,1.79
513,United Kingdom,2017,1.74
